<a href="https://colab.research.google.com/github/rajesh0305/Image-Processing-And-Computer-Vision-/blob/main/Dropout_%26_Regularizors%26Layers_Normalization%26Early_stoppingf_V1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install wandb
import wandb
from wandb.integration.keras import WandbMetricsLogger ,WandbModelCheckpoint
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import keras
from tensorflow import keras

In [20]:
sweep_config = {
    'method': 'grid',
'metric' : {
    'name': 'val_accuracy',
    'goal':'maximize'
},
    'parameters': {
         'batch_size' :{'values':[8]},
         'learning_rate' :{'values' :[0.01]},
         'hidden_nodes': {'values':[128]},
         'img_size': {'values':[16]},
         'epochs':{'values':[20]}
    }
}
sweep_id = wandb.sweep(sweep_config,project='5-flowers-new-v3')

Create sweep with ID: uhp86mfp
Sweep URL: https://wandb.ai/rajesh8368568776-national-institute-of-technology-karnat/5-flowers-new-V3/sweeps/uhp86mfp


In [21]:
def train():
  with wandb.init() as run:
    config = wandb.config

    # Constants
    IMG_HEIGHT = config.img_size
    IMG_WIDTH = config.img_size
    IMG_CHANNELS = 3
    CLASS_NAMES = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]

    def read_and_decode(filename, resize_dims):
        img_bytes = tf.io.read_file(filename)
        img = tf.image.decode_jpeg(img_bytes, channels=IMG_CHANNELS)
        img = tf.image.convert_image_dtype(img, tf.float32)
        img = tf.image.resize(img, resize_dims)
        return img

    def parse_csvline(csv_line):
        record_default = ["", ""]
        filename, label_string = tf.io.decode_csv(csv_line, record_default)

        img = read_and_decode(filename, [IMG_HEIGHT, IMG_WIDTH])

        # Convert label string to an integer index
        label = tf.where(tf.equal(CLASS_NAMES, label_string))[0, 0]

        return img, label

    # Define datasets
    train_dataset = (
        tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/train_set.csv")
        .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(config.batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    eval_dataset = (
        tf.data.TextLineDataset("gs://cloud-ml-data/img/flower_photos/eval_set.csv")
        .map(parse_csvline, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(config.batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )

    #architecture
    regularizers = tf.keras.regularizers.l1_l2(0,0.1)
    # model = keras.Sequential([
    #     keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),
    #     keras.layers.Dense(config.hidden_nodes,kernel_regularizer=regularizers, activation="relu"),
    #     keras.layers.Dropout(0.5), # means 50% of the neurons not used
    #     keras.layers.Dense(len(CLASS_NAMES), activation="softmax")
    # ])

    model = keras.Sequential([
        keras.layers.Flatten(input_shape=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)),

        keras.layers.Dense(config.hidden_nodes,kernel_regularizer=regularizers),
        keras.layers.BatchNormalization(),
        keras.layers.Activation("relu"),
        keras.layers.Dropout(0.5),

        keras.layers.Dense(len(CLASS_NAMES),kernel_regularizer = regularizers),
        keras.layers.BatchNormalization(),
        keras.layers.Activation("softmax")
    ])

    model.compile(
          optimizer="adam",
          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
          metrics=["accuracy"]
      )

    model.fit(
      train_dataset,
      validation_data=eval_dataset,
      epochs=config.epochs,
      callbacks=[WandbMetricsLogger(log_freq=5),
                 tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience = 3,restore_best_weights = True)
                 ]
      )

In [22]:
wandb.agent(sweep_id,function=train)

wandb: Agent Starting Run: pkigclzu with config:
wandb: 	batch_size: 8
wandb: 	epochs: 20
wandb: 	hidden_nodes: 128
wandb: 	img_size: 16
wandb: 	learning_rate: 0.01


Epoch 1/20


/usr/local/lib/python3.12/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


    413/Unknown 83s 195ms/step - accuracy: 0.3368 - loss: 10.2070

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


413/413 ━━━━━━━━━━━━━━━━━━━━ 94s 220ms/step - accuracy: 0.3369 - loss: 10.1951 - val_accuracy: 0.4216 - val_loss: 2.0363
Epoch 2/20
413/413 ━━━━━━━━━━━━━━━━━━━━ 85s 206ms/step - accuracy: 0.3771 - loss: 1.9282 - val_accuracy: 0.3757 - val_loss: 1.8923
Epoch 3/20
413/413 ━━━━━━━━━━━━━━━━━━━━ 89s 215ms/step - accuracy: 0.3733 - loss: 1.8562 - val_accuracy: 0.3973 - val_loss: 1.8521
Epoch 4/20
413/413 ━━━━━━━━━━━━━━━━━━━━ 142s 343ms/step - accuracy: 0.3737 - loss: 1.7871 - val_accuracy: 0.3919 - val_loss: 1.9521
Epoch 5/20
413/413 ━━━━━━━━━━━━━━━━━━━━ 89s 215ms/step - accuracy: 0.3967 - loss: 1.8094 - val_accuracy: 0.3865 - val_loss: 1.9794
Epoch 6/20
413/413 ━━━━━━━━━━━━━━━━━━━━ 84s 203ms/step - accuracy: 0.3863 - loss: 1.7805 - val_accuracy: 0.3973 - val_loss: 1.8261
Epoch 7/20
413/413 ━━━━━━━━━━━━━━━━━━━━ 83s 201ms/step - accuracy: 0.3807 - loss: 1.7676 - val_accuracy: 0.2189 - val_loss: 2.3089
Epoch 8/20
413/413 ━━━━━━━━━━━━━━━━━━━━ 82s 198ms/step - accuracy: 0.3853 - loss: 1.7580 - v

batch/accuracy,▁▁▃▃▅▆▆▆▅▅▅▅▅▆▃▆█▇▆▆▆▆▆▆▄▆▅▆███▇▇▄▆▅▅▂▂▅
batch/batch_step,▁▁▁▁▁▂▂▂▂▂▂▃▃▄▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇████
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▇▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,▁▅▆▆█▆▇▆▅▆▆▇▅▂
epoch/epoch,▁▂▂▃▃▄▄▅▅▆▆▇▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,█▆▇▇▇▇▁▃█▅█▅▄▆
epoch/val_loss,▅▃▂▃▄▂█▆▁▆▁▄▄▃
batch/accuracy,0.36131


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.
